In [1]:
!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [2]:
import pymysql

In [7]:
con = pymysql.connect(host='localhost' , port=3306 , user ='root', password = '1234', db='mldb',charset = 'utf8')
con
print('1. 연결성공')

1. 연결성공


In [10]:
cur = con.cursor()
cur
print('2. 커서 객체 획득')

2. 커서 객체 획득


In [12]:
sql= 'select * from member'
result = cur.execute(sql)
result 
print('3. sql전송 성공 ', result )

3. sql전송 성공  15


In [13]:
rows = cur.fetchall()
print('4. 검색결과 ------------')
print(rows)

4. 검색결과 ------------
(('alicia', 'a100', 'lim', '010'), ('apple', 'apple', 'apple', '555555555'), ('apple111', 'spring', 'hong', '111'), ('com', 'com', 'com', 'com'), ('park', 'park', 'park', '888'), ('pen', 'pen00', 'lim', '010'), ('spring', 'spring', 'spring', 'spring'), ('summer', 'summer', 'summer', 'summer'), ('sun', 'sun', 'sun', 'sun'), ('sunday2', 'sunday2', 'sunday2', 'sunday2'), ('water', 'w1000', 'park', '011'), ('win', 'win', 'win', 'win'), ('win2', 'win2', 'win2', 'win2'), ('win3', 'win3', 'win3', 'win3'), ('winter', 'winter', 'winter', 'winter'))


In [14]:
con.close()

In [18]:
for index, row in enumerate(rows) : 
    print(index, row[0])

0 alicia
1 apple
2 apple111
3 com
4 park
5 pen
6 spring
7 summer
8 sun
9 sunday2
10 water
11 win
12 win2
13 win3
14 winter


In [22]:
sql = """
SELECT m.name, m.tell, b.contant from board b 
inner join member m on m.id =b.writer 
WHERE content like '%java%' """

print(sql)


SELECT m.name, m.tell, b.contant from board b 
inner join member m on m.id =b.writer 
WHERE content like '%java%' 


In [23]:
def connect():
    con = pymysql.connect(host='localhost' , 
                          port=3306 , user ='root', 
                          password = '1234', db='mldb',charset = 'utf8')
    print('1.연결성공')
    cur=con.cursor()
    print('2. 커서객체 획득')
    return cur, con

In [25]:
cur2 , con =connect()
cur2

1.연결성공
2. 커서객체 획득


In [32]:
def sql_execute(cur2, data):
    sql = """
    select o2.orderid, m.name, o2.productid, o2.totalprice
    from orderlist o2
    inner join `member` m 
    on o2.memberid = m.id and m.id = %s 
    order by productid desc
    """
    result = cur2.execute(sql,data)
    return result 

In [47]:
sql_result = sql_execute(cur2,'park')
sql_result

4

In [48]:
who= input('누가?? >> ')
sql_result2 = sql_execute(cur2,who)
sql_result2 

누가?? >> park


4

In [42]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows 

In [49]:
result_rows = out_data(cur2)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000))

In [51]:
def db_close(con):
    con.close()

In [52]:
db_close(con)

In [53]:
## 제일 높은 금액은?

In [55]:
[x[3] for x in result_rows]

[6000, 5000, 10000, 3000]

In [58]:
max([x[3] for x in result_rows])

10000

In [59]:
## 제일 많이 산 물건의 id는?

In [60]:
import pandas as pd

In [70]:
result_df = pd.DataFrame(data= result_rows, columns=['주문id','name','제품id','주문총가격'])

In [71]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000


In [96]:
from collections import Counter

In [97]:
counter_id = Counter(result_df['제품id'])

In [98]:
counter_id

Counter({'101': 1, '100': 3})

In [99]:
max_id= counter_id.most_common(n=1)
# print(max_id)
# print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

'100'

In [100]:
def sql_excute2(cur2,data):
    sql="""
    select * from product
    where id =%s
    """
    result2= cur2.execute(sql,data)
    return result2

In [101]:
cur2, con =connect()
cur2

1.연결성공
2. 커서객체 획득


In [102]:
sql_result2= sql_excute2(cur2,max_id_result)
sql_result2

1

In [103]:
result_rows = out_data(cur2)
result_rows

(('100', 'java', 'fun java', '1000'),)

In [104]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('-----------------------------------------------')
print('물건의 이름은 ' , result_rows[0][1])
print('물건의 내용은 ' , result_rows[0][2])
print('물건의 가격은 ' , result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
-----------------------------------------------
물건의 이름은  java
물건의 내용은  fun java
물건의 가격은  1000


In [105]:
def sql_excute3(con,cur2 , data):
    sql = """
    insert into orderlist values (null,%s,%s,%s)
    """
    result2 = cur2.execute(sql,data)
    con.commit()
    return result2 

In [106]:
order_question = input('구매하시겠습니까? (yes/no)  ')
if order_question =='yes':
    print('-----구매를 진행합니다. ----------')
    your_id= input('당신의 id는 ')
    data_list=[your_id, result_rows[0][0],int(result_rows[0][3])]
    result3= sql_excute3(con,cur2,data_list)
    if result3==1:
        print('-------구매 완료---------')
    else:
        print('------구매 문제 발생')

구매하시겠습니까? (yes/no)  yes
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------


In [107]:
who= input('누가?? >> ')
sql_result2 = sql_execute(cur2,who)
sql_result2 

누가?? >> park


6

In [108]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows 

In [109]:
result_rows = out_data(cur2)
result_rows

((9, 'park', '109', 8000),
 (2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000),
 (8, 'park', '100', 1000))

In [110]:
db_close(con)

In [111]:
result_df2 = pd.DataFrame(data= result_rows, columns=['주문id','name','제품id','주문총가격'])

In [112]:
result_df2

,주문id,name,제품id,주문총가격
0,9,park,109,8000
1,2,park,101,6000
2,1,park,100,5000
3,3,park,100,10000
4,4,park,100,3000
5,8,park,100,1000


### mini project 


추천 알고리즘 : 사용자가 가장 최근 구매한 물품의 카테고리와 같은 제품을 추천한다.

In [202]:
def connect():
    con = pymysql.connect(host='localhost' , 
                          port=3306 , user ='root', 
                          password = '1234', db='mldb',charset = 'utf8')
    print('1.연결성공')
    cur=con.cursor()
    print('2. 커서객체 획득')
    return cur, con

In [203]:
cur2, con = connect()

1.연결성공
2. 커서객체 획득


In [204]:
def sql_execute4(cur3, data):
    sql = """
    select o2.orderid, m.name, o2.productid, p.category
    from orderlist o2
    inner join `member` m 
    on o2.memberid = m.id and m.id = %s 
    inner join product p 
    on o2.productid = p.id 
    order by productid desc
    """
    result = cur2.execute(sql,data)
    return result 

In [205]:
result = sql_execute4(cur, 'park')

In [206]:
result_rows = out_data(cur2)

In [207]:
result_rows

((9, 'park', '109', 4),
 (2, 'park', '101', 2),
 (1, 'park', '100', 1),
 (3, 'park', '100', 1),
 (4, 'park', '100', 1),
 (8, 'park', '100', 1))

In [208]:
df = pd.DataFrame(result_rows , columns=['orderid','이름','물건id','물건 카테고리'])
df

,orderid,이름,물건id,물건 카테고리
0,9,park,109,4
1,2,park,101,2
2,1,park,100,1
3,3,park,100,1
4,4,park,100,1
5,8,park,100,1


In [209]:
latest_id = df[df.orderid == max(df.orderid)]['물건 카테고리'].values
latest_id[0]

4

In [210]:
def sql_excute5(cur2,data):
    sql="""
    select * from product p
    where p.category = %s
    """
    result2= cur2.execute(sql,data)
    return result2

In [211]:
sql_excute2(cur2,latest_id[0])

3

In [212]:
recommend = out_data(cur2)
recommend

(('105', 'ios', 'fun ios', '6000', 4),
 ('109', 'python', 'good python', '10000', 4),
 ('112', 'python v2.0', 'old', '5000', 4))

In [213]:
len(recommend)

3

In [220]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('-----------------------------------------------')

for i in recommend :
    print('추천 제품 :', i[1], '\t 가격 : ',i[3])


당신에게 추천하고 싶은 물건은 다음과 같습니다.
-----------------------------------------------
추천 제품 : ios 	 가격 :  6000
추천 제품 : python 	 가격 :  10000
추천 제품 : python v2.0 	 가격 :  5000


In [179]:
def sql_excute6(con ,cur2 , data):
    sql = """
    insert into orderlist values (null,%s,%s,%s)
    """
    result2 = cur2.execute(sql,data)
    con.commit()
    return result2 

In [224]:
n=0
while n==0:
    order_question = input('몇번제품 구매하시겠습니까? (싫으면 n) >>> ')
    if order_question !='n':
        print('-----구매를 진행합니다. ----------')
        your_id= input('당신의 id는 ')
        
        data_list=[your_id, recommend[int(order_question)-1][0], int(recommend[int(order_question)-1][3])]
        result3= sql_excute3(con,cur2,data_list)
        if result3==1:
            print('-------구매 완료---------')
        else:
            print('------구매 문제 발생')
    else:
        n=1

몇번제품 구매하시겠습니까? (싫으면 n) >>> 1
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------
몇번제품 구매하시겠습니까? (싫으면 n) >>> 2
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------
몇번제품 구매하시겠습니까? (싫으면 n) >>> 3
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------
몇번제품 구매하시겠습니까? (싫으면 n) >>> n
